### WorkFlow

##### Problem Statement
##### Data Exploration
##### Data Cleaning
##### Feature Engineering
##### Predictive modelling

#### Import the following packages:

In [30]:
#Import the right libraries 
# load the libraries needed to run the analysis
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import re
import scipy
import pickle

#### Load the housing dataset

In [31]:
# Training data
house = pd.read_csv('train.csv')

# Test data 
house_test = pd.read_csv('test.csv')

trainSize = house.shape[0]
house.SalePrice = house.SalePrice.apply(np.log)
print(house.shape)
print(house_test.shape)

(1460, 81)
(1459, 80)


In [32]:
#Transpose the results to make printing on the screen easier.
summary = house.describe()
summary = summary.transpose()
print(summary)

                count          mean          std          min          25%  \
Id             1460.0    730.500000   421.610009     1.000000   365.750000   
MSSubClass     1460.0     56.897260    42.300571    20.000000    20.000000   
LotFrontage    1201.0     70.049958    24.284752    21.000000    59.000000   
LotArea        1460.0  10516.828082  9981.264932  1300.000000  7553.500000   
OverallQual    1460.0      6.099315     1.382997     1.000000     5.000000   
OverallCond    1460.0      5.575342     1.112799     1.000000     5.000000   
YearBuilt      1460.0   1971.267808    30.202904  1872.000000  1954.000000   
YearRemodAdd   1460.0   1984.865753    20.645407  1950.000000  1967.000000   
MasVnrArea     1452.0    103.685262   181.066207     0.000000     0.000000   
BsmtFinSF1     1460.0    443.639726   456.098091     0.000000     0.000000   
BsmtFinSF2     1460.0     46.549315   161.319273     0.000000     0.000000   
BsmtUnfSF      1460.0    567.240411   441.866955     0.000000   

#### Find correlation between different input features and the target:

In [4]:
# Plotting correlation 
#color map _ , ax = plt.subplots( figsize =( 12 , 10 ) ) 
#corr = df.corr(method='pearson') 
#cmap = sns.diverging_palette( 220 , 10 , as_cmap = True ) 
_# = sns.heatmap( corr, cmap = cmap, square=True, cbar_kws={ 'shrink' : .9 }, ax=ax, annot = True, annot_kws = { 'fontsize' : 12 })

SyntaxError: invalid syntax (<ipython-input-4-ef692026ddce>, line 2)

#### Some details about the data:

In [33]:
#Understanding Data 
df.describe() 

NameError: name 'df' is not defined

In [34]:
#save the Id column:
train_id = house['Id']
test_id  = house_test['Id']

# Now drop the 'Id' column 
# since we can not use it as a feature to train our model.
house.drop("Id", axis = 1, inplace = True)
house_test.drop("Id", axis = 1, inplace = True)

In [35]:
# distinguish the NA and missing in train data
miss = list(house.columns[house.isnull().sum(axis = 0) >= 1])
miss.remove('LotFrontage')
miss.remove('MasVnrArea')
miss.remove('GarageYrBlt')
miss.remove('MasVnrType')

house.loc[house['MasVnrType'].isnull() == True, 'MasVnrType'] = 'None'
house.loc[house['Electrical'].isnull() == True, 'Electrical'] = 'SBrkr'

for i in range(len(miss)):
    house.loc[house[miss[i]].isnull() == True, miss[i]] = 'NA'

In [36]:
with open('data_description.txt', 'r') as file:
    lines_level = file.readlines()

In [37]:
feature = ''
levels = {}
level = ''

for line in lines_level:
    if (not not re.findall(': ', line)) and (not re.findall('story:', line)):
#         features.append(re.split(':', line)[0])
        feature = re.split(':', line)[0]
        levels[feature] = []
    else:
        if line != '\n':
#             levels.append(re.split('\t', line)[0].strip())
            level = re.split('\t', line)[0].strip()
            if level != '': levels[feature].append(level)
levels['Id'] = []
levels['SalePrice'] = []
# print(levels)

In [38]:
# find ordinal and non ordinal
with open('DataDocumentation.txt', 'r',encoding='cp1252') as file:
    lines = file.readlines()

In [39]:
# Convert string data to numerical data 

nom_lines = []
ord_lines = []
for line in lines:
    if re.findall('(Nominal)', line):
        nom_lines.append(re.sub(' ', '', re.split('\(Nominal\)', line)[0].strip()))
    elif re.findall('(Ordinal)', line):
        ord_lines.append(re.sub(' ', '', re.split('\(Ordinal\)', line)[0].strip()))
print(nom_lines)
print(ord_lines)

['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LandContour', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1', 'Exterior2', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']
['LotShape', 'Utilities', 'LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence']


In [40]:
def my_cat(col, levels, ord_lines):
    if not not levels[col.name]: 
        if col.name in ord_lines:
            temp = pd.Categorical(list(col.astype('str')), ordered=True, categories=levels[col.name][::-1])
            temp.name = col.name
            return temp
        else:
            temp = pd.Categorical(list(col.astype('str')), categories=levels[col.name][::-1])
            temp.name = col.name
            return temp
   
    else:
        return col

In [41]:
for col in house:
    house[col] = my_cat(house[col], levels, ord_lines)

In [42]:
miss = list(house_test.columns[house_test.isnull().sum(axis = 0) >= 1])
miss.remove('LotFrontage')
miss.remove('MasVnrArea')
miss.remove('GarageYrBlt')
miss.remove('TotalBsmtSF')
miss.remove('BsmtHalfBath')
miss.remove('BsmtFullBath')
miss.remove('GarageCars')
miss.remove('GarageArea')
miss.remove('BsmtFinSF1')
miss.remove('BsmtFinSF2')
miss.remove('BsmtUnfSF')

house_test.loc[house_test['MasVnrType'].isnull() == True, 'MasVnrType'] = 'CBlock'
house_test.loc[house_test['SaleType'].isnull() == True, 'SaleType'] = 'Oth'
house_test.loc[house_test['Functional'].isnull() == True, 'Functional'] = 'Sal'
house_test.loc[house_test['MSZoning'].isnull() == True, 'MSZoning'] = 'A'
house_test.loc[house_test['Exterior1st'].isnull() == True, 'Exterior1st'] = 'WdShing'
house_test.loc[house_test['Exterior2nd'].isnull() == True, 'Exterior2nd'] = 'Wd Shng'

for i in range(len(miss)):
    house_test.loc[house_test[miss[i]].isnull() == True, miss[i]] = 'NA'

In [43]:
for col in house_test:
    house_test[col] = my_cat(house_test[col], levels, ord_lines)

In [44]:
print(levels['OverallQual'][::-1])
print(levels['ExterQual'][::-1])
print(levels['BsmtQual'][::-1])
print(levels['KitchenQual'][::-1])
print(levels['GarageFinish'][::-1])
print(levels['FireplaceQu'][::-1])
print(levels['HeatingQC'][::-1])

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
['Po', 'Fa', 'TA', 'Gd', 'Ex']
['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
['Po', 'Fa', 'TA', 'Gd', 'Ex']
['NA', 'Unf', 'RFn', 'Fin']
['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
['Po', 'Fa', 'TA', 'Gd', 'Ex']


In [45]:
def OrdinalsToNumerics(all_data):    
    all_data['ExterQual'] = all_data['ExterQual'].map(lambda x: {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}.get(x, 0))
    all_data['BsmtQual'] = all_data['BsmtQual'].map(lambda x: {'NA':1,'Po':2, 'Fa':3, 'TA':4, 'Gd':5, 'Ex':6}.get(x, 0))
    all_data['KitchenQual'] = all_data['KitchenQual'].map(lambda x: {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}.get(x, 0))
    all_data['GarageFinish'] = all_data['GarageFinish'].map(lambda x: {'NA':1, 'Unf':2, 'RFn':3, 'Fin':4}.get(x, 0))
    all_data['FireplaceQu'] = all_data['FireplaceQu'].map(lambda x: {'NA':1,'Po':2, 'Fa':3, 'TA':4, 'Gd':5, 'Ex':6}.get(x, 0))
    all_data['HeatingQC'] = all_data['HeatingQC'].map(lambda x: {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}.get(x, 0))

    all_data['OverallQual'] = pd.to_numeric(all_data['OverallQual'])
    all_data['ExterQual'] = pd.to_numeric(all_data['ExterQual'])
    all_data['BsmtQual'] = pd.to_numeric(all_data['BsmtQual'])
    all_data['KitchenQual'] = pd.to_numeric(all_data['KitchenQual'])
    all_data['GarageFinish'] = pd.to_numeric(all_data['GarageFinish'])
    all_data['FireplaceQu'] = pd.to_numeric(all_data['FireplaceQu'])
    all_data['HeatingQC'] = pd.to_numeric(all_data['HeatingQC'])

In [46]:
# Idea taken from this script: https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
# # Add some family features directly to new columns in the dataset

def FeatureEngineering(all_data):
    # Adding total sqfootage feature 
    all_data['GarageYrBlt'][all_data['GarageYrBlt'].isnull()] = all_data['YearBuilt'][all_data['GarageYrBlt'].isnull()]
    all_data['GarageYrBlt'] = all_data['GarageYrBlt'] - all_data['YearBuilt']

    #all_data['YrSold'] = all_data['YrSold'] - all_data['YearBuilt']
    
    all_data['MoSold'] = pd.Categorical(list(all_data['MoSold'].astype('str')))
    
    all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

    # GarageYrBlt, GarageArea and GarageCars : Replacing missing data with 0 (Since No garage = no cars in such garage.)
    for col in ('GarageArea', 'GarageCars'):
        all_data[col] = all_data[col].fillna(0)

    # BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath and BsmtHalfBath : missing values are likely zero for having no basement
    for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
        all_data[col] = all_data[col].fillna(0)

    # LotFrontage : Since the area of each street connected to the house property most likely have a similar 
    # area to other houses in its neighborhood , we can fill in missing values by the median LotFrontage of the neighborhood.
    # Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
    all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
    
    all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
    
    # Utilities : For this categorical feature all records are "AllPub", except for one "NoSeWa" and 2 NA . Since the house with 'NoSewa' is in the training set, this feature won't help in predictive modelling. We can then safely remove it.
    all_data = all_data.drop(['Utilities'], axis=1)

drop_cols1 = ["Alley", "ExterCond", "BsmtFinSF2", "Heating", "Electrical",\
              "BsmtHalfBath", "PavedDrive", "Fence", "MiscFeature", "MoSold"]

drop_cols2 = ["BsmtFinType2", "LandContour", "EnclosedPorch",\
              "SaleType", "Functional", "LotConfig",\
              "ScreenPorch", "YrSold", "LandSlope"]


In [47]:
# Specify columns to drop
def DropColumns(all_data):
    for col in [*drop_cols1, *drop_cols2]:
        all_data.drop([col], axis=1, inplace=True)

In [48]:
price = house.SalePrice
house.drop('SalePrice', axis=1, inplace=True)

In [49]:
OrdinalsToNumerics(house)
OrdinalsToNumerics(house_test)

FeatureEngineering(house)
FeatureEngineering(house_test)

DropColumns(house)
DropColumns(house_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [50]:
all_data = pd.concat([house, house_test], ignore_index = True)
all_data = pd.get_dummies(all_data, drop_first=True)

In [51]:
train_clean = all_data.iloc[:trainSize,:]
train_clean = train_clean.assign(SalePrice = price.values)
train_clean.shape

(1460, 219)

In [52]:
trainSize

1460

In [53]:
test_clean = all_data.iloc[trainSize:, :]
test_clean.shape

(1459, 218)

## Machine learning Models

In [54]:
#Load the libraries

# The usual
import numpy as np

# sklearn tools for model training and assesment
import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV, train_test_split,KFold
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor #as gbr as rfr
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

from mlxtend.regressor import StackingRegressor
from xgboost.sklearn import XGBRegressor
# LightGBM
import lightgbm as lgb
# Lightxgboost
import xgboost as xgb

In [55]:
#Create a function to handle rmse computation
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

#### Split the dataset into training and testing

In [59]:
# Prepare the data sets
test_ratio_const = 0.2
train_data   = train_clean.drop('SalePrice', axis=1)
train_target = train_clean[['SalePrice']]
test_data    = test_clean

# Split the data into training and testing datasets 
X_train, X_test, y_train, y_test = train_test_split(train_data, train_target,\
                                                    test_size=test_ratio_const)

In [60]:
print('X train shape = ',X_train.shape)
print('X test shape = ', X_test.shape)
print('Y train shape = ', y_train.shape)
print('Y test shape = ',y_test.shape)

X train shape =  (1168, 218)
X test shape =  (292, 218)
Y train shape =  (1168, 1)
Y test shape =  (292, 1)


In [61]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_std = scaler.transform(X_train)
X_test_std  = scaler.transform(X_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  import sys


In [62]:
kfold = KFold(n_splits=5, shuffle=True, random_state=80)

### 1. Linear Regression Model

#### Building the model

In [344]:
model_lr = linear_model.LinearRegression()

#### Fitting the model

In [345]:
model_lr.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

#### Evaluating the model and printing the results

In [346]:
print("\n#### Performance ####") 
print("RMSE train: {}".format(rmse(y_train, model_lr.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_lr.predict(X_test))))


#### Performance ####
RMSE train: 0.10262489450225752
RMSE test : 0.1273096943564471


### 2. Kernel Ridge Regression 

#### Creating a list of the hyperparameters

In [347]:
grid_param = [{'alpha': np.logspace(-4, 4, 20)}]

#### Buiding the model

In [350]:
#run the grid search
grid_search = GridSearchCV(estimator=linear_model.Ridge,cv=kfold, param_grid=grid_param)

#### Fitting the model

In [351]:
gs.fit(X_train_std, y_train)

Fitting 5 folds for each of 552960 candidates, totalling 2764800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   46.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.8min


KeyboardInterrupt: 

#### Evaluating the model and printing the results

In [64]:
print('Best params: {}'.format(gs.best_params_))
print('Best score : {}'.format(gs.best_score_))


model_rd = gs.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_rd.predict(X_train_std))))
print("RMSE test : {}".format(rmse(y_test,  model_rd.predict(X_test_std))))

Best params: {'alpha': 545.5594781168514}
Best score : 0.862816990220294
RMSE train: 0.11500081056347171
RMSE test : 0.09967169272824544


In [ ]:
grid_param = [{'alpha': 545.55}]

In [ ]:
#run the grid search
grid_search = GridSearchCV(estimator=linear_model.Ridge,cv=kfold, param_grid=grid_param)

### 3. Lasso Regression

#### Creating a list of hyperparameters

In [71]:
grid_param = [{'alpha': np.logspace(-3, 4, 20)}] 

#### Buiding the model

In [72]:
gs = GridSearchCV(estimator=linear_model.Lasso(random_state=rs_const, normalize=False), param_grid=grid_param, cv=5)

#### Fitting the model

In [73]:
gs.fit(X_train_std, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=80,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'alpha': array([1.00000e-03, 2.33572e-03, 5.45559e-03, 1.27427e-02, 2.97635e-02,
       6.95193e-02, 1.62378e-01, 3.79269e-01, 8.85867e-01, 2.06914e+00,
       4.83293e+00, 1.12884e+01, 2.63665e+01, 6.15848e+01, 1.43845e+02,
       3.35982e+02, 7.84760e+02, 1.83298e+03, 4.28133e+03, 1.00000e+04])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

#### Evaluating the model and printing the results

In [74]:
print('Best params: {}'.format(gs.best_params_))
print('Best score : {}'.format(gs.best_score_))

model = gs.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model.predict(X_train_std))))
print("RMSE test : {}".format(rmse(y_test,  model.predict(X_test_std))))

Best params: {'alpha': 0.00545559478116852}
Best score : 0.8564711534949876
RMSE train: 0.11402145996484198
RMSE test : 0.09659128729814824


### 4. Elastic Net Regression

#### Creating a list of the hyperparameters

In [86]:
grid_param = [{'alpha': np.logspace(-2, 4, 20), 'l1_ratio': np.linspace(0.015, 1, 20)}]

#### Buidling the model

In [87]:
gs = GridSearchCV(estimator=linear_model.ElasticNet(random_state=rs_const), param_grid=grid_param, cv=5)

#### Fitting the model

In [88]:
gs.fit(X_train_std, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=80, selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'alpha': array([1.00000e-02, 2.06914e-02, 4.28133e-02, 8.85867e-02, 1.83298e-01,
       3.79269e-01, 7.84760e-01, 1.62378e+00, 3.35982e+00, 6.95193e+00,
       1.43845e+01, 2.97635e+01, 6.15848e+01, 1.27427e+02, 2.63665e+02,
       5.45559e+02, 1.12884e+03, 2.33572e+03, 4.83293e+03, 1.0... 0.53342, 0.58526, 0.63711, 0.68895,
       0.74079, 0.79263, 0.84447, 0.89632, 0.94816, 1.     ])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

#### Evaluating the model and printing the results

In [89]:
print('Best params: {}'.format(gs.best_params_))
print('Best score : {}'.format(gs.best_score_))

model_en = gs.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_en.predict(X_train_std))))
print("RMSE test : {}".format(rmse(y_test,  model_en.predict(X_test_std))))

Best params: {'alpha': 0.18329807108324356, 'l1_ratio': 0.015}
Best score : 0.8652189389746141
RMSE train: 0.1127655445071562
RMSE test : 0.0955844475998484


### 5. Gradient Boost

#### Create a list of the hyperparameters

In [53]:
#grid_params = {
 #'max_features' : ['sqrt'],
 #'max_depth' : [3,5,7],
 #'min_samples_leaf' : [5,10,15,20],                           
 #'min_samples_split': [5,10,15,20],                                  
 #'n_estimators': [1000,2000,3000,4000],
 #'learning_rate' : [0.001, 0.005, 0.01,0.05,0.01],
 #'loss' : ['huber','ls'],
 #'subsample':[0.6,0.7,0.8,0.9]}

In [54]:
#cvx = GradientBoostingRegressor(random_state=rs_const)

In [82]:
#gs = GridSearchCV(estimator=GradientBoostingRegressor(random_state=rs_const), param_grid=grid_params,n_jobs=-1, cv=5,verbose=True)

In [83]:
#gs.fit(X_train, y_train)

In [84]:
#Best bgt
model_gbt = GradientBoostingRegressor(\
                                    learning_rate=0.005,\
                                    n_estimators=3000,\
                                    max_depth=4,\
                                    min_samples_split=10,\
                                    min_samples_leaf=15,\
                                    max_features='sqrt',\
                                    subsample=0.8,\
                                    loss = 'huber',\
                                    random_state=rs_const)
model_gbt.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.005, loss='huber', max_depth=4,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=3000,
             n_iter_no_change=None, presort='auto', random_state=80,
             subsample=0.8, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

### 6. Light Gradient Boosting

#### Create a list of the hyperparameters

In [120]:
# Define the parameter grid 
grid_params = {'num_leaves':[2,5,7],
'learning_rate':[0.001, 0.005, 0.01, 0.05, 0.1], 
'n_estimators':[500,600,700,800],
'max_bin': [50,60,70], 
'bagging_fraction': [0.6, 0.7, 0.8, 0.9],
'bagging_freq': [3, 5, 7, 9], 
'feature_fraction': [0.2, 0.3, 0.4, 0.5],
'feature_fraction_seed':[2,4,9,10], 
'bagging_seed': [4,9,11],
'min_sum_hessian_in_leaf': [1,11,20],
'min_data_in_leaf':[4,6,8,10],
}

In [186]:
#grid_params = {'num_leaves':[4,5], 
#'learning_rate':[0.05,0.06], 
#'n_estimators':[720,800],
#'max_bin': [50,55,56], 
#'bagging_fraction': [0.78,0.8],
#'bagging_freq': [4,5], 
#'feature_fraction': [0.2319],
#'feature_fraction_seed':[8,9], 
#'bagging_seed': [8,9],
#'min_sum_hessian_in_leaf': [10,11],
#'min_data_in_leaf':[5,6],
#}
#Best score : 0.8940731040429333
#RMSE train: 0.07270665730916882
#RMSE test : 0.10623077886089849

In [299]:
grid_params = {'num_leaves':[4,5,6], 
'learning_rate':[0.04,0.05,0.06], 
'n_estimators':[720,721],
'max_bin': [54,55,56], 
'bagging_fraction': [0.7,0.8],
'bagging_freq': [4,5,6], 
'feature_fraction': [0.1,0.2319,0.3],
'feature_fraction_seed':[9], 
'bagging_seed': [9],
'min_sum_hessian_in_leaf': [11],
'min_data_in_leaf':[5,6,7]
}

In [300]:
#set up the parameter nthread for XGBoost parallelisation 
cvx = lgb.LGBMRegressor(random_state=rs_const, objective= "Regression",nthread=-1)

In [301]:
grid_search=GridSearchCV(estimator=cvx,param_grid=grid_params,n_jobs=-1,cv=5, verbose=True)

In [302]:
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 2916 candidates, totalling 14580 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   26.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed: 20.8min
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed: 25.6min
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed: 30.3min
[Parallel(n_jobs=-1)]: Done 9792 tasks      | elapsed: 35.3min
[Parallel(n_jobs=-1)]: Done 11242 tasks      |

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, nthread=-1, num_leaves=31,
       objective='Regression', random_state=80, reg_alpha=0.0,
       reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'num_leaves': [4, 5, 6], 'learning_rate': [0.04, 0.05, 0.06], 'n_estimators': [720, 721], 'max_bin': [54, 55, 56], 'bagging_fraction': [0.7, 0.8], 'bagging_freq': [4, 5, 6], 'feature_fraction': [0.1, 0.2319, 0.3], 'feature_fraction_seed': [9], 'bagging_seed': [9], 'min_sum_hessian_in_leaf': [11], 'min_data_in_leaf': [5, 6, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None,

In [303]:
print('Best params: {}'.format(grid_search.best_params_))
print('Best score : {}'.format(grid_search.best_score_))

model_lgb = grid_search.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_lgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_lgb.predict(X_test))))

Best params: {'bagging_fraction': 0.8, 'bagging_freq': 6, 'bagging_seed': 9, 'feature_fraction': 0.3, 'feature_fraction_seed': 9, 'learning_rate': 0.05, 'max_bin': 54, 'min_data_in_leaf': 5, 'min_sum_hessian_in_leaf': 11, 'n_estimators': 720, 'num_leaves': 5}
Best score : 0.8945234683899054
RMSE train: 0.07769224378017096
RMSE test : 0.10735421329669734


In [304]:
grid_params = {'num_leaves':[5], 
'learning_rate':[0.05], 
'n_estimators':[720],
'max_bin': [54], 
'bagging_fraction': [0.8],
'bagging_freq': [6], 
'feature_fraction': [0.3],
'feature_fraction_seed':[9], 
'bagging_seed': [9],
'min_sum_hessian_in_leaf': [10,11],
'min_data_in_leaf':[5]
}

In [ ]:
Best params: {'bagging_fraction': 0.8, 'bagging_freq': 5, 'bagging_seed': 9, 'feature_fraction': 0.1, 'feature_fraction_seed': 9, 'learning_rate': 0.06, 'max_bin': 56, 'min_data_in_leaf': 6, 'min_sum_hessian_in_leaf': 11, 'n_estimators': 720, 'num_leaves': 6}
Best score : 0.8926095278770132
RMSE train: 0.073467283820668
RMSE test : 0.10763867443207041

In [309]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.3,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 10)

In [310]:
print('Best params: {}'.format(grid_search.best_params_))
print('Best score : {}'.format(grid_search.best_score_))

model_lgb = grid_search.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_lgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_lgb.predict(X_test))))

Best params: {'bagging_fraction': 0.8, 'bagging_freq': 6, 'bagging_seed': 9, 'feature_fraction': 0.3, 'feature_fraction_seed': 9, 'learning_rate': 0.05, 'max_bin': 54, 'min_data_in_leaf': 5, 'min_sum_hessian_in_leaf': 11, 'n_estimators': 720, 'num_leaves': 5}
Best score : 0.8945234683899054
RMSE train: 0.07769224378017096
RMSE test : 0.10735421329669734


In [290]:
#set up the parameter nthread for XGBoost parallelisation 
cvx = lgb.LGBMRegressor(random_state=rs_const, objective= "Regression",nthread=-1)

In [291]:
grid_search=GridSearchCV(estimator=cvx,param_grid=grid_params,n_jobs=-1,cv=5, verbose=True)

In [292]:
grid_search.fit(X_train,y_train)
#with open('lgb_hpred.pickle','wb') as f:
    #pickle.dump(grid_search,f)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   34.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  1.8min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, nthread=-1, num_leaves=31,
       objective='Regression', random_state=80, reg_alpha=0.0,
       reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'num_leaves': [4, 5, 6], 'learning_rate': [0.04, 0.05], 'n_estimators': [720, 721], 'max_bin': [54, 55, 56], 'bagging_fraction': [0.8], 'bagging_freq': [5], 'feature_fraction': [0.1, 0.2319, 0.3], 'feature_fraction_seed': [9], 'bagging_seed': [9], 'min_sum_hessian_in_leaf': [11], 'min_data_in_leaf': [6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [293]:
print('Best params: {}'.format(grid_search.best_params_))
print('Best score : {}'.format(grid_search.best_score_))

model_lgb = grid_search.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_lgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_lgb.predict(X_test))))

Best params: {'bagging_fraction': 0.8, 'bagging_freq': 5, 'bagging_seed': 9, 'feature_fraction': 0.1, 'feature_fraction_seed': 9, 'learning_rate': 0.04, 'max_bin': 56, 'min_data_in_leaf': 6, 'min_sum_hessian_in_leaf': 11, 'n_estimators': 720, 'num_leaves': 6}
Best score : 0.8923620736452994
RMSE train: 0.08301808005402683
RMSE test : 0.10470213710463049


In [ ]:
grid_params = {'num_leaves':[4,5,6],
'learning_rate':[0.001, 0.005, 0.01], 
'n_estimators':[600,700,800],
'max_bin': [50,60], 
'bagging_fraction': [0.6, 0.7, 0.8, 0.9],
'bagging_freq': [3, 5, 7, 9], 
'feature_fraction': [0.2, 0.3, 0.4, 0.5],
'feature_fraction_seed':[2,4,9,10], 
'bagging_seed': [4,9,11],
'min_sum_hessian_in_leaf': [10,11],
'min_data_in_leaf':[4,6,8,10],
}

In [245]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.24,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [247]:
print('Best params: {}'.format(grid_search.best_params_))
print('Best score : {}'.format(grid_search.best_score_))

model_lgb = grid_search.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_lgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_lgb.predict(X_test))))

Best params: {'bagging_fraction': 0.8, 'bagging_freq': 5, 'bagging_seed': 9, 'feature_fraction': 0.2319, 'feature_fraction_seed': 9, 'learning_rate': 0.05, 'max_bin': 55, 'max_depth': 30, 'min_data_in_leaf': 6, 'min_sum_hessian_in_leaf': 11, 'n_estimators': 720, 'num_leaves': 5}
Best score : 0.8874534326944281
RMSE train: 0.07849660243523138
RMSE test : 0.10432902919105366


#### Evaluate the performance of the regressor and print the metrics:

In [155]:
# Evaluate performance of Support Vector Regressor 
print('Best params: {}'.format(gs.best_params_))
print('Best score : {}'.format(gs.best_score_))

model_lgb = gs.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_lgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_lgb.predict(X_test))))

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
# Create Support Vector Regression model 
sv_regressor = SVR(kernel='linear', C=1.0, epsilon=0.1) 
 
# Train Support Vector Regressor 
sv_regressor.fit(X_train, y_train) 

In [ ]:
# Evaluate performance of Support Vector Regressor 
y_test_pred = sv_regressor.predict(X_test) 
mse = mean_squared_error(y_test, y_test_pred) 
evs = explained_variance_score(y_test, y_test_pred)  
print("\n#### Performance ####") 
print("Mean squared error =", round(mse, 2)) 
print("Explained variance score =", round(evs, 2)) 

#### Print the performance metrics:

In [ ]:
# Measure performance 
print("Linear Regressor performance:") 
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, y_test_pred), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(y_test, y_test_pred), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(y_test, y_test_pred), 2)) 
print("Explained variance score =", round(sm.explained_variance_score(y_test, y_test_pred), 2)) 
print("R2 score =", round(sm.r2_score(y_test, y_test_pred), 2)) 

### Let's visualize the input data using a scatter plot:

In [ ]:
# Visualize input data 
plt.figure() 
plt.scatter(class_0[:, 0], class_0[:, 1], s=75, facecolors='black',  
            edgecolors='black', linewidth=1, marker='x') 
plt.scatter(class_1[:, 0], class_1[:, 1], s=75, facecolors='white',  
            edgecolors='black', linewidth=1, marker='o') 
plt.title('Input data') 

#### Evaluate the performance of the classifier by printing the classification report:

### 5. XGB

#### Setting up the model hyperparameters

In [87]:
params = {
 'max_depth':[4,6,8],
 'n_estimators':[1000,1500,2000,2500,3000],
'min_child_weight':range(1,3),
'learning_rate':[.1,.01,.001],
'colsample_bytree':[.8,.9,1]
,'gamma':[0,1]}

#### Buidling the model

In [ ]:
#set up the parameter nthread for XGBoost parallelisation 
cvx = xgb.XGBRegressor(random_state=rs_const, objective= "reg:linear",nthread=-1)

#### Fitting the model

In [ ]:
grid_search=GridSearchCV(estimator=cvx,param_grid=params,n_jobs=-1,cv=5, verbose=True)

#### Evaluating the model and printing the results

In [97]:
gs.fit(X_train,y_train)

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 40.2min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 114.1min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 223.2min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 341.4min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 525.9min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 700.5min
[Parallel(n_jobs=-1)]: Done 2700 out of 2700 | elapsed: 771.6min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=-1, objective='reg:linear', random_state=80,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [4, 6, 8], 'n_estimators': [1000, 1500, 2000, 2500, 3000], 'min_child_weight': range(1, 3), 'learning_rate': [0.1, 0.01, 0.001], 'colsample_bytree': [0.8, 0.9, 1], 'gamma': [0, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [98]:
print('Best params: {}'.format(grid_search.best_params_))
print('Best score : {}'.format(grid_search.best_score_))

model_xgb = grid_search.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

Best params: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 4, 'min_child_weight': 2, 'n_estimators': 2000}
Best score : 0.8945528519113021
RMSE train: 0.056005723679234946
RMSE test : 0.09890042608186725


In [90]:
#Best xgb
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.01,\
                                    max_depth=4,\
                                    min_child_weight=4,\
                                    n_estimators=2000,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=4, min_child_weight=4, missing=None, n_estimators=2000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=80,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

### 6. Random Forest

#### Create a list of the hyperparameters

In [91]:
grid_params = {
 'max_features':[50,70,90],
 'n_estimators':[150,200,250],
'min_samples_split':[2,4,6],
'min_samples_leaf':[1,2]}

#### Building the model

In [48]:
gs = GridSearchCV(estimator=RandomForestRegressor(random_state=rs_const), param_grid=grid_params,n_jobs=-1, cv=5,verbose=True)

#### Fitting the model

In [49]:
gs.fit(X_train_std, y_train)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   46.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  3.1min finished
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=80, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_features': [50, 70, 90], 'n_estimators': [150, 200, 250], 'min_samples_split': [2, 4, 6], 'min_samples_leaf': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

#### Applying the tuned hyperparameters to the model and printing the results

In [95]:
#Best rf
model_rf = RandomForestRegressor(\
            max_features=70,\
            n_estimators=200,\
            min_samples_split=2,\
            min_samples_leaf=1,\
            random_state=rs_const,\
            n_jobs=-1)

model_rf.fit(X_train, y_train)
print("RMSE train: {}".format(rmse(y_train, model_rf.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_rf.predict(X_test))))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


RMSE train: 0.053199962181578714
RMSE test : 0.11588640719406174


#### Compute the relative feature importance:

In [ ]:
# Extract feature importances 
feature_importances = regressor.feature_importances_ 
feature_names = housing_data.feature_names 

#### Sort them so that they can be plotted:

In [ ]:
# Sort the values and flip them 
index_sorted = np.flipud(np.argsort(feature_importances)) 

#### Plot the bar graph:

In [ ]:
# Plot the bar graph 
plt.figure() 
plt.bar(pos, feature_importances[index_sorted], align='center') 
plt.xticks(pos, feature_names[index_sorted]) 
plt.ylabel('Relative Importance') 
plt.title('Feature importance using AdaBoost regressor') 
plt.show() 

In [ ]:
# Extremely Random Forests regressor 
params = {'n_estimators': 100, 'max_depth': 4, 'random_state': 0} 
regressor = ExtraTreesRegressor(**params) 
regressor.fit(X_train, y_train) 

In [428]:
from sklearn.neural_network import MLPRegressor

In [431]:
model_nn = MLPRegressor(hidden_layer_sizes = (90, 90),alpha = 2.75)
model_nn.fit(X_train,y_train)
#y_pred_nn = model_nn.predict(X_train)
#score_nn = np.sqrt(mean_squared_error(y_train, y_pred_nn))
#print("Multi-layer Perceptron Score(On Training DataSet) :",score_nn)

#y_pred_nn_test = model_nn.predict(X_test)
#y_pred_nn_test = np.exp(y_pred_nn_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1316: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPRegressor(activation='relu', alpha=2.75, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(90, 90), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [432]:
#print('Best params: {}'.format(grid_search.best_params_))
#print('Best score : {}'.format(grid_search.best_score_))

#model_nn = grid_search.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model.predict(X_test))))

RMSE train: 0.10203058682861088
RMSE test : 0.09562801364724414


In [ ]:
RMSE train: 0.10203058682861088
RMSE test : 0.09562801364724414

In [361]:
#Network Parameters
m = len(X_train)
n = 3 # Number of features
n_hidden = 20 # Number of hidden neurons

In [433]:
from keras.models import Model 
# Hyperparameters
batch = 20
eta = 0.01
max_epoch = 100
# Build Model 
model = Sequential() 
model.add(Dense(n_hidden, input_dim=n, activation='relu')) 
model.add(Dense(1, activation='sigmoid')) 
model.summary() 
# Summarize the model 
#Compile model 
model.compile(loss='mean_squared_error', optimizer='adam') 
#Fit the model 
model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=max_epoch, batch_size=batch, verbose=1) 
#Predict the values and calculate RMSE and R2 score 
y_test_pred = model.predict(X_test) 
y_train_pred = model.predict(X_train) 
r2 = r2_score( y_test, y_test_pred ) 
rmse = mean_squared_error( y_test, y_test_pred ) 
print( "Performance Metrics R2 : {0:f}, RMSE : {1:f}".format( r2, rmse ) )

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/imp.py", line 342, in load_dynamic
    return _load(s

In [64]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,CSVLogger
from tensorflow.keras.utils import plot_model

ImportError: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/imp.py", line 342, in load_dynamic
    return _load(s

#### Predict the output:

In [ ]:
def parameterChoosing(self):
        #Set the parameters by cross-validation
        tuned_parameters = [{'max_depth': range(20,60),
                             'n_estimators': range(10,40),
                             'max_features': ['sqrt', 'log2', None]
                             }
                            ]

        clf = GridSearchCV(RandomForestRegressor(n_estimators=30), tuned_parameters, cv=5, scoring='mean_squared_error')
        clf.fit(self.X_train, self.y_train.ravel())

        print "Best parameters set found on development set:\n"
        print clf.best_params_

        print "Grid scores on development set:\n"
        for params, mean_score, scores in clf.grid_scores_:
            print "%0.3f (+/-%0.03f) for %r\n" % (mean_score, scores.std() * 2, params)

        print "MSE for test data set:\n"
        y_true, y_pred = self.y_test, clf.predict(self.X_test)
        print mean_squared_error(y_true, y_pred) 

In [65]:
import tensorflow as tf
from tensorflow.keras.models       import Sequential, load_model
from tensorflow.keras.layers       import Dense,Dropout
from tensorflow.keras.callbacks    import ModelCheckpoint, EarlyStopping,CSVLogger
from tensorflow.keras.utils        import plot_model

ImportError: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/imp.py", line 342, in load_dynamic
    return _load(s